In [96]:
import numpy as np
import pandas as pd
import matplotlib as plt

In [97]:
df=pd.read_csv('Iris.csv')
#normalisation
df.fillna(df.mean(),inplace=True)

C:\Users\bt3222\AppData\Local\Temp\ipykernel_6888\1350022555.py:3: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.fillna(df.mean(),inplace=True)


In [98]:
df['Species']=df['Species'].map({'Iris-setosa':0,'Iris-versicolor':1,'Iris-virginica':2}).astype('int64')

from sklearn.model_selection import train_test_split
x1_train,x1_test,y1_train,y1_test=train_test_split(df.iloc[:,1:4],df.iloc[:,5],test_size=0.2,random_state=0)

In [99]:
def entropy(y):
    proportions = np.bincount(y) / len(y)
    return -np.sum([p * np.log2(p) for p in proportions if p > 0])

def information_gain(X_column, y, threshold):
    left_mask = X_column <= threshold
    right_mask =X_column>threshold

    if len(y[left_mask]) == 0 or len(y[right_mask]) == 0:
        return 0

    parent_entropy = entropy(y)
    n = len(y)
    n_left, n_right = len(y[left_mask]), len(y[right_mask])

    e_left = entropy(y[left_mask])
    e_right = entropy(y[right_mask])

    child_entropy = (n_left / n) * e_left + (n_right / n) * e_right
    return parent_entropy - child_entropy

def best_split(X, y):
    best_gain = -1
    best_feature, best_threshold = None, None

    for feature_idx in range(X.shape[1]):
        thresholds = np.unique(X.iloc[:, feature_idx].values)
        for threshold in thresholds:
            gain = information_gain(X.iloc[:, feature_idx].values, y, threshold)

            if gain > best_gain:
                best_gain = gain
                best_feature = feature_idx
                best_threshold = threshold

    return best_feature, best_threshold

def build_tree(X, y, depth=0, max_depth=None):
    if len(np.unique(y)) == 1 or (max_depth is not None and depth >= max_depth):
        return np.bincount(y).argmax()

    feature, threshold = best_split(X, y)
    if feature is None:
        return Counter(y).most_common(1)[0][0]

    left_mask = X.iloc[:, feature].values <= threshold
    right_mask = X.iloc[:, feature].values > threshold

    left_subtree = build_tree(X[left_mask], y[left_mask], depth + 1, max_depth)
    right_subtree = build_tree(X[right_mask], y[right_mask], depth + 1, max_depth)

    return {"feature": feature, "threshold": threshold, "left": left_subtree, "right": right_subtree}

def predict_sample(x, tree):
    if not isinstance(tree, dict):
        return tree
    feature, threshold = tree["feature"], tree["threshold"]
    feature_value = x[feature]
    if feature_value <= threshold:
        return predict_sample(x, tree["left"])
    else:
        return predict_sample(x, tree["right"])
    
def predict(x,tree):
    return np.array([predict_sample(inp,tree) for inp in x.values])

In [100]:
tree=build_tree(x1_train,y1_train,0,4)

In [101]:
tree

{'feature': 2,
 'threshold': 1.7,
 'left': 0,
 'right': {'feature': 2,
  'threshold': 4.7,
  'left': {'feature': 0,
   'threshold': 4.9,
   'left': {'feature': 1, 'threshold': 2.4, 'left': 1, 'right': 2},
   'right': 1},
  'right': {'feature': 2,
   'threshold': 5.1,
   'left': {'feature': 0, 'threshold': 5.8, 'left': 2, 'right': 2},
   'right': 2}}}

In [102]:
p=predict(x1_test,tree)
p


array([2, 1, 0, 2, 0, 2, 0, 1, 2, 1, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 2, 1,
       2, 0, 2, 0, 1, 1, 1, 0], dtype=int64)

In [103]:
x1_test.iloc[2][0]

5.5

In [104]:
type(x1_test)

pandas.core.frame.DataFrame

In [105]:
val=0
for i in range(len(p)):
    if (p[i]==y1_test.iloc[i]):
        val+=1
print(f"accuracy: {val/len(y1_test)*100}")

accuracy: 90.0
